In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Load the Iris dataset
iris = load_iris()
X = iris['data']
y = iris['target']

# One-hot encode the target labels
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.5, random_state=2)

# Create the model
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=5, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


2024-09-13 13:32:58.353473: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 13:32:58.396447: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 13:32:58.396976: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 13:32:59.242397: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/50
15/15 [==============================] - 0s 2ms/step - loss: 1.2528 - accuracy: 0.1733
Epoch 2/50
15/15 [==============================] - 0s 2ms/step - loss: 1.2032 - accuracy: 0.1867
Epoch 3/50
15/15 [==============================] - 0s 2ms/step - loss: 1.1588 - accuracy: 0.2133
Epoch 4/50
15/15 [==============================] - 0s 2ms/step - loss: 1.1172 - accuracy: 0.2267
Epoch 5/50
15/15 [==============================] - 0s 2ms/step - loss: 1.0815 - accuracy: 0.2667
Epoch 6/50
15/15 [==============================] - 0s 2ms/step - loss: 1.0468 - accuracy: 0.3333
Epoch 7/50
15/15 [==============================] - 0s 2ms/step - loss: 1.0095 - accuracy: 0.4267
Epoch 8/50
15/15 [==============================] - 0s 2ms/step - loss: 0.9697 - accuracy: 0.5733
Epoch 9/50
15/15 [==============================] - 0s 2ms/step - loss: 0.9265 - accuracy: 0.6267
Epoch 10/50
15/15 [==============================] - 0s 1ms/step - loss: 0.8891 - accuracy: 0.6667
Epoch 11/50
15/15 [

In [2]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
y_keras = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))))

3/3 [==============================] - 0s 2ms/step
Accuracy: 0.8266666666666667


In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Iris dataset
iris = load_iris()
X = iris['data']
y = iris['target']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=2)

# Create the C file
with open('Iris_testdata/test_dataset.c', 'w') as f:
    # Write the data samples array
    f.write("float test_data[%d][%d] = {\n" % (X_test.shape[0], X_test.shape[1]))
    for row in X_test:
        f.write("    {%s},\n" % ", ".join(f"{x:.6f}f" for x in row))
    f.write("};\n\n")

    # Write the labels array
    f.write("int test_labels[%d] = {\n" % len(y_test))
    f.write("    %s\n" % ", ".join(map(str, y_test)))
    f.write("};\n")

print("Test dataset has been saved to 'test_dataset.c'")


Test dataset has been saved to 'test_dataset.c'


In [18]:
# Get the weights and biases
layer1_weights = model.layers[0].get_weights()[0].T
layer1_bias = model.layers[0].get_weights()[1]
layer2_weights = model.layers[1].get_weights()[0].T
layer2_bias = model.layers[1].get_weights()[1]

# Write weights and biases to weights.cpp
with open('Iris_testdata/weights.cpp', 'w') as f:
    f.write('#include <vector>\n\n')
    
    f.write('std::vector<std::vector<float>> layer1_weights = {\n')
    for row in layer1_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer1_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer1_bias))
    f.write('};\n\n')

    f.write('std::vector<std::vector<float>> layer2_weights = {\n')
    for row in layer2_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer2_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer2_bias))
    f.write('};\n')

print("Weights and biases have been saved to 'weights.cpp'")


Weights and biases have been saved to 'weights.cpp'


In [19]:
!g++ -std=c++11 Iris_testdata/test.cpp -o Iris_testdata/test_nn

In [20]:
!./Iris_testdata/test_nn

Input size: 4
Neuron 0 output: 1.78267
Neuron 1 output: 0.216063
Neuron 2 output: 0.624568
Neuron 3 output: 2.15343
Neuron 4 output: 0
Neuron 5 output: 0.911524
Neuron 6 output: 0.224269
Neuron 7 output: 0
Layer 1 complete
Output 0: 2.57988
Output 1: -1.77073
Output 2: -2.89633
Layer 2 complete
Softmax output 0: 0.983203
Softmax output 1: 0.0126822
Softmax output 2: 0.00411486
Input size: 4
Neuron 0 output: 1.6616
Neuron 1 output: 0.502544
Neuron 2 output: 0.277075
Neuron 3 output: 2.15729
Neuron 4 output: 0
Neuron 5 output: 1.184
Neuron 6 output: 0.639714
Neuron 7 output: 0
Layer 1 complete
Output 0: 2.34257
Output 1: -1.09197
Output 2: -2.98949
Layer 2 complete
Softmax output 0: 0.964251
Softmax output 1: 0.0310878
Softmax output 2: 0.00466127
Input size: 4
Neuron 0 output: 0
Neuron 1 output: 0.399697
Neuron 2 output: 0
Neuron 3 output: 0
Neuron 4 output: 1.10425
Neuron 5 output: 0.491003
Neuron 6 output: 1.16628
Neuron 7 output: 1.58097
Layer 1 complete
Output 0: -2.7115
Output 1: 2